In [2]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [13]:
env = gym.make("FrozenLake-v0")

[2020-02-20 16:41:06,766] Making new env: FrozenLake-v0


In [14]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [15]:
print(action_space_size)
print(state_space_size)
print(env.observation_space)
print(env.action_space)

4
16
Discrete(16)
Discrete(4)


In [16]:
print(env.P[1][0])

[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True)]


## Value Iteration

In [20]:
def value_iteration(env, gamma = 1.0):
    value_table = np.zeros(env.observation_space.n)
    no_of_iterations = 10000
    threshold = 1e-20
    for i in range(no_of_iterations):
        updated_value_table = np.copy(value_table)
        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]:
                    tans_prob, next_state, reward_prob, _ = next_sr
                    next_states_rewards.append((tans_prob*(reward_prob + gamma*updated_value_table[next_state])))
                    Q_value.append(np.sum(next_states_rewards))
            # Pick up the maximum Q value and update it as value of a state
            value_table[state] = max(Q_value)
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
            print('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return value_table

In [21]:
env = gym.make("FrozenLake-v0")
optimal_value_function = value_iteration(env)

[2020-02-20 16:41:48,885] Making new env: FrozenLake-v0


Value-iteration converged at iteration# 1373.


In [22]:
optimal_value_function

array([0.82352941, 0.82352941, 0.82352941, 0.82352941, 0.82352941,
       0.        , 0.52941176, 0.        , 0.82352941, 0.82352941,
       0.76470588, 0.        , 0.        , 0.88235294, 0.94117647,
       0.        ])

## Policy Iteration

After finding the optimal value function, We have to extract our optima policy from it.

In [23]:
def extract_policy(value_table, gamma = 1.0):
    policy = np.zeros(env.observation_space.n)
    for state in range(env.observation_space.n):
        Q_table = np.zeros(env.action_space.n)
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]:
                trans_prob, next_state, reward_prob, _ = next_sr
                Q_table[action] += (trans_prob*(reward_prob + gamma*value_table[next_state]))
        policy[state] = np.argmax(Q_table)
    return policy
            

In [25]:
optimal_policy = extract_policy(optimal_value_function,1.0)

In [26]:
optimal_policy

array([0., 3., 3., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

In [38]:
env.P[0][0]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

In [27]:
def compute_value_function(policy, gamma = 1.0):
    value_table = np.zeros(env.nS)
    threshold = 1e-10
    while True:
        updated_value_table = np.copy(value_table)
        for state in range(env.nS):
            action = policy[state]
            value_table[state] = sum([trans_prob*(reward_prob + gamma*updated_value_table[next_state])
                                 for trans_prob, next_state, reward_prob, _ in env.P[state][action]])
        if (np.sum(np.fabs(updated_value_table))<= threshold):
            break
    return value_table

In [28]:
random_policy = np.zeros(env.observation_space.n)
new_value_function = compute_value_function(random_policy,1.0)

In [29]:
new_value_function

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
new_policy = extract_policy(new_value_function, 1.0)
new_policy

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [31]:
def policy_iteration(env,gamma = 1.0):
    random_policy = np.zeros(env.observation_space.n)
    no_of_iterations = 200000
    for i in range(no_of_iterations):
        new_value_function = compute_value_function(random_policy,gamma)
        new_policy = extract_policy(new_value_function,gamma)
        if (np.all(random_policy == new_policy)):
            print('Policy-Iteration converged at step %d.' %(i+1))
            break
        random_policy = new_policy
    return new_policy

In [32]:
optimal_policy = policy_iteration(env, gamma = 1.0)
optimal_policy

Policy-Iteration converged at step 3.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.])

In [34]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

In [35]:
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0
    for step in range(max_steps_per_episode): 

    # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

In [37]:
print(action)

1
